In [1]:
from config import *
import numpy as np
import pandas as pd

In [2]:
# (...,70](70,72],(72,74],(74,76],(76,78],(78,80],(80,..)
def groupAge(age):
    if age <= 70: return 0
    if age > 80: return 6
    return (age - 70 + 1) // 2

def groupAct(act, act_avg):
    return act <= act_avg

def is_number(a):
    try:
        a = float(a)
    except:
        return False
    return not np.isnan(a)

In [3]:
# Id,Longevity,Education,Sex,Age,Pet,Children,Region,Activity,
# MedExam,Sport
# ==>
# Longevity,Education,Sex,Age,Pet,Children,Activity,
# Sport,Family,IsAlone,Age*Education,Status

def processing(df):
    df = df.drop(columns=['Id', 'Region', 'MedExam'])

    df['Sex'] = train_df['Sex'].map(lambda a: 1 if a == 'male' else 0).astype(int)

    idmax = df['Age'].value_counts().idxmax()
    df['Age'] = df['Age'].map(lambda v: v if is_number(v) else idmax).astype(int)

    act_avg = int(df['Activity'].mean())
    df['Sport'] = df['Activity'].map(lambda a: groupAct(a, act_avg)).astype(int)
                          
    df['Family'] = df['Pet'] + df['Children']
    df['IsAlone'] = df['Family'].map(lambda f: 1 if not f else 0).astype(int)
    
    df['Age*Education'] = df['Age'].map(lambda a: groupAge(a)) * df['Education']
    df['Status'] = df['Age'].map(lambda a: groupAge(a)) + df['Sport'] + df['IsAlone']
    
    return df

In [5]:
train_df = pd.read_csv(train_raw)
test_df = pd.read_csv(test_raw)

train_df = processing(train_df)
test_df = processing(test_df)

train_df.to_csv(train_processed, index=False)
test_df.to_csv(test_processed, index=False)

In [6]:
train_df.head()

,Longevity,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone,Age*Education,Status
0,0,3,0,73,1,0,725,1,1,0,6,3
1,1,1,1,75,1,0,7128,0,1,0,3,3
2,1,3,1,73,0,0,793,1,0,1,6,4
3,1,1,1,74,1,0,5310,0,1,0,2,2
4,0,3,0,74,0,0,805,1,0,1,6,4
